# Displaying options

The display behaviour of **Gnuplot.jl** depends on the value of the `Gnuplot.options.gpviewer` boolean option:

- if `true` the plot is displayed in a gnuplot window, using one of the interactive terminals such as `wxt`, `qt` or `aqua`.  There is exactly one window for each session, and the plots are updated by replacing the displayed image.  The preferred terminal can optionally be set using `Gnuplot.options.term`;

- if `false` the plot is displayed through the Julia [multimedia interface](https://docs.julialang.org/en/v1/base/io-network/#Multimedia-I/O-1), i.e. it is exported as either a `png`, `svg` or `html` file, and displayed in an external viewer.  In this case the package is unable to replace a previous plot, hence each update results in a separate image being displayed.  The terminal options to export the images are set in `Gnuplot.options.mime`.

The latter approach can only be used when running a Jupyter, JupyterLab or Juno session, while the former approach is appropriate in all cases (most notably, for the standard Julia REPL).  The `Gnuplot.options.gpviewer` flag is automatically set when the package is first loaded according to the runtime environment, however the user can change its value at any time to fit specific needs.  The following examples show the difference between the two approaches. 

Recall that the `:-` symbol has a special meaning in **Gnuplot.jl**:

- if given as first argument to a `@gp` or `@gsp` call, it append data and commands to a previously started plot.  To start a new plot omit `:-` as first arugment;
- if given as last argument it avoids updating the current plot, although data and commands are saved to be used at a later time.  To actually update a plot omit `:-` as last arugment.

The `:-` symbol has been chosen since it reminds of hyphenation in natural language, and allow to split `@gp` and `@gsp` calls into multiple, possibly non-contiguous, statements.


## Display plots in a REPL session
Even if the following examples run in a notebook we can simulate a REPL session by explicitly setting `Gnuplot.options.gpviewer` to `true`:

In [1]:
using Gnuplot
Gnuplot.options.gpviewer = true; # simulate usage in a REPL

# Optionally, we can set a custom terminal for interactive use
Gnuplot.options.term = "wxt size 800,400";

# Terminate all sessions
Gnuplot.quitall()

# Generate some numbers to plot
x = 1:0.1:10
y1 = sin.(x)
y2 = cos.(x)
y3 = y1.+y2;

Start a new plot using the `y1` dataset:

In [2]:
@gp x y1 "w l t 'sin'" :-

As expected, nothing is displayed since we used the `:-` symbol as last argument to `@gp`.
Now add a second dataset (`y2`), and omit the trailing `:-` symbol:

In [3]:
@gp :- x y2 "w l t 'cos'"

A new window should popup showing the plot with two datasets.  Now add a third dataset (`y3`):

In [4]:
@gp :- x y3 "w l t 'sin+cos' lw 2";

The plot has been updated by replacing the window content. The presence of the semicolon has no effect here, and the same applies when we involke `display()` on the returned value: 

In [5]:
display(ans)  # nothing happens here

Now insert the same `@gp` calls into a (possibly very long and complex) function, and invoke it:

In [6]:
function complex_function()
    # ...
    x = 1:0.1:10
    y1 = sin.(x)
    y2 = cos.(x)
    y3 = y1.+y2;
    @gp    :dummy x y1 "w l t 'sin'" :-
    @gp :- :dummy x y2 "w l t 'cos'"
    @gp :- :dummy x y3 "w l t 'sin+cos' lw 2";
    #...
    return nothing
end

complex_function()

Here we used a separate session called `dummy` to show the plot in a separate window.  Note that the same code, used in the REPL or within a function, generates the same plot.  Also note that the plot is updated two times since there is no trailing `:-` symbol on both lines 8 and 9.  We could slightly improve the performances by adding a trailing `:-` to line 8, and display the final plot in line 9 (although in this case the plot is too simple to produce any measurable improvement).


## Display plots in an external viewer

Let's repeat the above examples redirecting the images on this notebook:

In [7]:
Gnuplot.options.gpviewer = false;  # use Julia multimedia interface

#=
Gnuplot.jl export images using the following MIME types:
- image/png
- image/svg+xml
- text/html
and the corresponding terminal options are set through the `Gnuplot.options.mime` dictionary.

Jupyter would store images in all such formats in the notebook.  To shrink the notebook size
you may choose the format providing best results, and disable the others:
=#
Gnuplot.options.mime[MIME"image/png"] = "";
Gnuplot.options.mime[MIME"image/svg+xml"] = "";
Gnuplot.options.mime[MIME"text/html"] = "svg enhanced standalone mouse size 800,400";

In [8]:
@gp x y1 "w l t 'sin'" :-

Again, the trailing `:-` symbol avoids creating a plot.  If we omit it:

In [9]:
@gp :- x y2 "w l t 'cos'"

we obtain an inline plot with two datasets. Now the third dataset:

In [10]:
@gp :- x y3 "w l t 'sin+cos' lw 2";

According to the Julia convention, no plot is shown here since we added a trailing `;`, however all data have already been sent to gnuplot.

To display the updated plot we need an explicit call to `display()` on the returned value:

In [11]:
display(ans)

Unlike the previous case, note that the updated a plot now appears in a **separate** image.

The above code produced two inline images.  Now invoke the same code within the `complex_function()` function:

In [12]:
complex_function()

Unlike the previous case nothing appeared here since we need to explicitly invoke `display()` on the proper session:

In [13]:
display(@gp :dummy)

Clearly, the plot appears in the same notebook, even if the session is different.

## Summary

The following tables summarize the main aspects of the approaches discussed above:


| `Gnuplot.options.gpviewer = true`                                   | REPL, Jupyter or Juno
| :-------------                                                      | :-------------:
| Plots are shown                                                     | in dedicated window(s)
| Updating a plot                                                     | updates window content
| Output of different session goes to                                 | separate windows
| How many simultaneous plots can be shown?                           | all windows fitting the screen
| Calls to `display()` / automatic display (trailing `;`)             | does nothing / not applicable
| The top level code works also within a function?                    | yes, explicit `display()` calls are not required
| Terminal options are specified in                                   | `Gnuplot.options.term`


| `Gnuplot.options.gpviewer = false`                                  | REPL                           |   Jupyter                                               |    Juno
| :-------------                                                      | :-------------:                | :------------:                                          | :-------------:
| Plots are shown                                                     | not shown                      |  as inline images in the notebook                       |  as image in the plot pane
| Updating a plot                                                     | not shown                      | creates a separate image                                |  overwrites the plot pane content
| Output of different session goes to                                 | not shown                      |  the same notebook                                      |  the plot pane
| How many simultaneous plots can be shown?                           | not shown                      |  all those fitting in the visible part of the notebook  |  only one
| Calls to `display()` / automatic display (trailing `;`)             | not applicable                 |  updates the plot / applicable                          |  updates the plot / applicable
| The top level code works also within a function?                    | not applicable                 |  no, explicit `display()` calls are required            |  no, explicit `display()` calls are are required
| Terminal options are specified in                                   | not applicable                 |  `Gnuplot.options.mime`                                 |  `Gnuplot.options.mime`
